### 도쿄 부동산 사건 데이터 수집 
출처 :  <a href = 'http://www.oshimaland.co.jp'> 오시마랜드 </a>

사이트 정보 : 본 웹사이트는 자살사건, 시체유기사건, 살인사건 등 과거에 불미스러운 사건이 있었던 건물을 모아서 보여줍니다.

도쿄 23구와 다마 지역 (26개의 도시) 데이터 수집 할 예정이다 

특별도 각 구청 : https://good-luck-day.com/tokyo-district-list/
```
도쿄 23구 

    가쓰시카구(葛飾区)
    고토구(江東区)
    기타구(北区)
    나카노구(中野区)
    네리마구(練馬区)
    다이토구(台東区)
    도시마구(豊島区)
    미나토구(港区)
    메구로구(目黒区)
    스미다구(墨田区)
    스기나미구(杉竝区)
    신주쿠구(新宿区)
    세타가야구(世田谷区)
    시나가와구(品川区)
    시부야구(渋谷区)
    아다치구(足立区)
    아라카와구(荒川区)
    에도가와구(江戸川区)
    이타바시구(板橋区)
    오타구(大田区)
    분쿄구(文京区)
    주오구(中央区)
    지요다구(千代田区)

다마 지역

    고가네이시(小金井市)
    고다이라시(小平市)
    고마에시(狛江市)
    고쿠분지시(国分寺市)
    구니타치시(国立市)
    기요세시(清瀬市)
    니시토쿄시(西東京市)
    다마시(多摩市)
    다치카와시(立川市)
    마치다시(町田市)
    무사시노시(武藏野市)
    무사시무라야마시(武藏村山市)
    미타카시(三鷹市)
    아키루노시(あきる野市)
    아키시마시(昭島市)
    오메시(青梅市)
    이나기시(稲城市)
    조후시(調布市)
    하무라시(羽村市)
    하치오지시(八王子市)
    후추시(府中市)
    훗사시(福生市)
    히가시무라야마시(東村山市)
    히가시야마토시(東大和市)
    히가시쿠루메시(東久留米市)
    히노시(日野市)

```




In [ ]:
# !pip install webdriver-manager
# !pip install requests

In [1]:
import pandas as pd
import numpy as np 

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import time

In [2]:
#1. oshimaland 웹사이트 불러오기
def loadWeb(driver, url):
    # header 추가하기 

    
    # 화면 최대 크기 설정 
    driver.get(url)
    driver.maximize_window()
    #모든 창이 로딩 될떄까지 기다려 줌 
    time.sleep(3)
    #팝업창 화면 전환 후 닫아주기
    multi_window = driver.window_handles
    if(len(multi_window)>1):
        driver.switch_to.window(driver.window_handles[-1])
        driver.close()
        time.sleep(3)
        # 접근 페이지 다시 요청
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(url)
    
    # 전체 지도 축소 
    # driver.execute_script("document.body.style.zoom='60%'")    
    # time.sleep(2)




In [11]:
from selenium.webdriver.common.action_chains import ActionChains

from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException

def get_fire(driver, tokyo_data_dict, district):
        # fire 객체 수집 
        fire_object_list = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > div > div.mk-annotation-container')
        fire_list = fire_object_list.find_elements(By.CSS_SELECTOR, '.map-fire')
        print('fire 개수: ', len(fire_list))
        for fire in fire_list:
            try: 
                is_enabled = fire.is_enabled()
                actions = ActionChains(driver)                 
                if is_enabled:
                    # 클릭이 가능하며 현재 화면에 표시 될 때 클릭 수행
                    # print('click OK')
                    if fire.is_displayed():
                        # print('display OK')
                        # x_offset = 200  # x 좌표를 조정할 값
                        # y_offset = 200  # y 좌표를 조정할 값
                        # driver.execute_script("arguments[0].scrollIntoView();", fire)
                        # actions.move_to_element_with_offset(fire, x_offset, y_offset)
                        # actions.click()
                        # actions.perform()

                        # 팝업창 클릭                         
                        actions = ActionChains(driver)                 
                        actions.move_to_element(fire).click().perform()

                        # 팝업 정보 가져오기 
                        # time.sleep(5)
                        WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.property-info-date > span')))
                        info_date = driver.find_element(By.CSS_SELECTOR, 'li.property-info-date > span').text
                        info_address = driver.find_element(By.CSS_SELECTOR, 'li.property-info-address').text
                        info_content = driver.find_element(By.CSS_SELECTOR, 'div.popup-property-info > section > ul > li:nth-child(3)').text

                        tokyo_data_dict.append({
                            'district': district,
                            'date': info_date,
                            'address': info_address,
                            'content': info_content
                        })

                        time.sleep(3)

                    else:
                        pass
                        # print('display NOT')
                else:
                     pass
                    # print('click not')

            except (NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException) as e:
                # 에러 처리
                print(f"Error occurred: {e}")

                # 필요한 경우 WebDriver 인스턴스 재초기화 등의 작업 수행
                # driver = initialize_driver()

                continue


In [4]:
# info_data = driver.find_element(By.CSS_SELECTOR, 'li.property-info-date > span').text
# info_address = driver.find_element(By.CSS_SELECTOR, 'li.property-info-address').text
# info_content = driver.find_element(By.CSS_SELECTOR, 'div.popup-property-info > section > ul > li:nth-child(3)').text


In [5]:
from tqdm import tqdm, tqdm_notebook

tokyo_data_dict = []

def search_fire(driver, jp_district_list):
    for district in tqdm(jp_district_list):
        print(district)
        region_search_tab = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
        # 검색어 clear
        region_search_tab.clear()
        # 지역 검색
        region_search_tab.send_keys(district)
        time.sleep(2)
        # 검색 버튼 클릭 
        search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
        search_btn.send_keys(Keys.ENTER)
        # ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (751, 3641)
        time.sleep(8)
        # fire 객체 정보 수집 
        get_fire(driver,tokyo_data_dict, district)





In [6]:
# !pip install googletrans==4.0.0-rc1

#### 영어로된 일본 지역 리스트 본어로 전처리 
- 오시마랜드는 영오도 지원되나 일본어로 검색하는게 정확도가 높다 판단하여 일본어로 변경해준다 
- googletrans를 이용해 번역할 경우 정확도가 떨어진다
- 45개 지역 하드코딩으로 번역

In [7]:
from googletrans import Translator

# main_district = pd.read_csv('../jieun/main_district_df2.csv')
# district_list  = main_district['neighbourhood_cleansed'].values
# translator = Translator()
# jp_district_list = list(map(lambda district: translator.translate(district, src='en', dest='ja').text.replace(' ',''),district_list))
# jp_district_list[: 10]


In [8]:
jp_district_list = [
    '新宿区', '台東区', '墨田区', '豊島区', '渋谷区',
    '港区', '世田谷区', '大田区', '中野区', '中央区',
    '北区', '葛飾区', '杉並区', '江東区', '江戸川区',
    '文京区', '荒川区', '板橋区', '品川区', '千代田区',
    '足立区', '目黒区', '練馬区', '府中市', '八王子市',
    '日野市', '国分寺市', '町田市', '青梅市', '調布市',
    '武蔵野市', 'あきる野市', '三鷹市', '小金井市', '東村山市',
    '多摩市', '国立市', '狛江市', '西東京市', '立川市',
    '小平市', '羽村市', '武蔵村山市', '奥多摩町', '昭島市',
    '福生市'
]

In [9]:
from selenium import webdriver
 
url = 'https://www.oshimaland.co.jp/'

# chrome 이용 
#  WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No node with given id found"}
# chrome_option = webdriver.ChromeOptions()
# driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_option)

# firefox 이용 
driver = webdriver.Firefox()
loadWeb(driver, url)

#### 발생 에러들

1. WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No node with given id found"} : Chrome driver 문제
2. StaleElementReferenceException: Message: The element with the reference b3212e1d-3be9-4d7a-9bd2-9f736fc42e4a is stale; either its node document is not the active document, or it is no longer connected to the DOM
3. AttributeError: 'NoneType' object has no attribute 'text'
4. MoveTargetOutOfBoundsException

In [12]:
search_fire(driver, jp_district_list)

  0%|          | 0/46 [00:00<?, ?it/s]

新宿区
fire 개수:  180
Error occurred: Message: The element with the reference c825cfe5-1bf2-459b-802c-fe4810188868 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 422a8c95-2de5-40b6-ac99-d9cba498ddea is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 2cf3ffa6-5d6e-44dc-b2a4-11d3ae28fb52 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 8081fbc4-ddf0-4157-af7a-f39bc7219a12 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 1e98a237-809d-4e06-8633-87ee1b15b56b is stale; either its node document is not the active document, or it is no longer connected to the DO

  2%|▏         | 1/46 [04:06<3:04:34, 246.10s/it]

Error occurred: Message: The element with the reference 1fa5deba-5b4b-4884-8500-4791f1a9a350 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference d7fe28ec-fee8-4739-ae85-a48822e23b60 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 44ce10a7-7008-4abf-ab67-4cc6aa6e749a is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 1933f479-d2c0-4f66-9abb-5c4f96b8f714 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 5dcd2b29-7c11-4d5a-ad17-3e27b94b691f is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred:

  4%|▍         | 2/46 [04:58<1:37:02, 132.34s/it]

...

fire 개수:  0
昭島市
fire 개수:  8


 98%|█████████▊| 45/46 [25:20<00:15, 15.55s/it]

福生市
fire 개수:  15


100%|██████████| 46/46 [25:37<00:00, 33.43s/it]

Error occurred: Message: The element with the reference e62442c6-cd17-4e43-8c23-2c1bc9e8c50f is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference ab9fe42b-d212-4ed2-a4e0-d93990185287 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 72bb58c8-f623-4ead-bf68-b6834098b6d1 is stale; either its node document is not the active document, or it is no longer connected to the DOM



In [15]:
driver.quit()

In [13]:
tokyo_df = pd.DataFrame(tokyo_data_dict)
tokyo_df

,district,date,address,content
0,新宿区,平成24年11月5日,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見
1,新宿区,平成24年11月5日,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見
2,新宿区,令和4年,東京都新宿区西新宿一丁目3-9,飛び降り自殺
3,新宿区,,,
4,新宿区,平成20年1月5日,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見
...,...,...,...,...
298,羽村市,2005,東京都羽村市緑ヶ丘五丁目1-50,駐車スペース横の部屋で住民男性が首吊り自殺
299,武蔵村山市,不明,東京都羽村市羽中一丁目3-37,告知事項
300,昭島市,平成24年,東京都武蔵村山市中央一丁目24-3ハウスオブブーケA 1階,死体発見
301,福生市,令和2年9月7日,東京都昭島市田中町一丁目34-9,火災による死亡


In [14]:
tokyo_df.to_csv('selenium_tokyo_data.csv')

In [19]:
df = pd.read_csv('selenium_tokyo_data.csv', index_col=0)
df

,district,date,address,content
0,新宿区,平成24年11月5日,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見
1,新宿区,平成24年11月5日,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見
2,新宿区,令和4年,東京都新宿区西新宿一丁目3-9,飛び降り自殺
3,新宿区,NaN,NaN,NaN
4,新宿区,平成20年1月5日,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見
...,...,...,...,...
298,羽村市,2005,東京都羽村市緑ヶ丘五丁目1-50,駐車スペース横の部屋で住民男性が首吊り自殺
299,武蔵村山市,不明,東京都羽村市羽中一丁目3-37,告知事項
300,昭島市,平成24年,東京都武蔵村山市中央一丁目24-3ハウスオブブーケA 1階,死体発見
301,福生市,令和2年9月7日,東京都昭島市田中町一丁目34-9,火災による死亡


#### 지도 내 줌인 줌아웃 실패 

In [ ]:
# from selenium.webdriver.common.action_chains import ActionChains
# # https://stackoverflow.com/questions/64474653/how-to-webscrape-a-canvas-element-with-python-selenium
# canvas = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > canvas.rt-root')
# action = ActionChains(driver)

# map_canvas = driver.find_element_by_css_selector("div.mk-map-view > canvas.rt-root")

# from selenium.webdriver.common.action_chains import ActionChains

# # 지도 요소 위에서 마우스를 움직임
# actions = ActionChains(driver)
# actions.move_to_element(map_canvas).click().perform()

# actions.move_by_offset(0, 200).perform() # 줌인만 가능 

# #줌인 줌아웃 반복 
# action.move_to_element(canvas).move_by_offset(0, 0).click_and_hold().perform()
# action.move_by_offset(-200,0).release().perform()

#### Request API 이용한 방법 

In [ ]:
import json
from  bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import urllib.request as req
from tqdm import tqdm, tqdm_notebook
import time
import requests

In [ ]:
#TODO shinjuku ku test
search_region = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
search_region.send_keys(jp_district_list[20])
time.sleep(2)
# 검색 버튼 클릭 
search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
search_btn.send_keys(Keys.ENTER)

time.sleep(2)

In [ ]:
json_url = 'https://api.oshimaland.co.jp/map'
headers = {
    "User-Agent": "Mozilla/5.0",
    "Origin": "https://www.oshimaland.co.jp"
}
data = {
    # 'keys': ["1330021123012133", "1330021123012311", "1330021123013022", "1330021123013200"]
    'keys' : ['1330021123013021',  '1330021123013023',  '1330021123013201']
}

response = requests.post(json_url, headers=headers, json=data)
json_data = response.json()

In [ ]:
from itertools import chain

origin = list(data.values())
flatten_list = list(chain(*origin))
flatten_list

In [ ]:
jsonValueList = []
for idx in range(len(flatten_list)):
    for idx2 in range(len(json_data['markers'][flatten_list[idx]])):
        jsonValueList.append(json_data['markers'][flatten_list[idx]][idx2]['key'])

In [ ]:
getDataList2 = []
for idx in tqdm_notebook(range(len(jsonValueList))):
    json_url = f'https://www.oshimaland.co.jp/d/{jsonValueList[idx]}.json'
    url = req.Request(json_url, headers={"User-Agent": "Mozilla/5.0"})
    code = req.urlopen(url)
    soup = BeautifulSoup(code, 'html.parser')
    json_data = json.loads(soup.text)
    getDataList2.append({
        'info' : json_data['info'],
        'address' : json_data['ad'],
        'dt' : json_data['dt'],
        'cr' : json_data['cr'],
     })